<a href="https://colab.research.google.com/github/soulayman-git/test-sub-branch/blob/master/SystemeRecommandation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Systeme de recomandation des Films**

Algorithme de recherche BFS

In [6]:
import pandas as pd
from collections import defaultdict, deque
import re

# Charger les fichiers
movies_file = "drive/MyDrive/movies_reduced_to_500.xlsx"  # Chemin vers la table réduite des films
ratings_file = "drive/MyDrive/ratings_reduced_to_500.xlsx"  # Chemin vers la table réduite des ratings

# Lire les fichiers Excel
movies_data = pd.read_excel(movies_file, engine="openpyxl")
ratings_data = pd.read_excel(ratings_file, engine="openpyxl")

# Fonction pour extraire le titre sans l'année
def extraire_titre_sans_annee(titre):
    match = re.match(r"^(.*?)(?:\s\(\d{4}\))?$", titre)
    return match.group(1).strip() if match else titre.strip()

# Étape 1 : Réduire la base de données
def reduire_base_donnees(movies_data, ratings_data):
    """
    Combine les films et leurs notes pour créer une base cohérente.
    """
    popularite = ratings_data.groupby('movieId').agg(
        average_rating=('rating', 'mean'),
        num_ratings=('rating', 'count')
    ).reset_index()

    films_populaires = movies_data.merge(popularite, on='movieId')
    films_populaires['normalized_title'] = films_populaires['title'].apply(extraire_titre_sans_annee)
    return films_populaires[['movieId', 'title', 'genres', 'average_rating', 'num_ratings', 'normalized_title']]

# Réduire la base de données
films_reduits = reduire_base_donnees(movies_data, ratings_data)
print(f"Base réduite générée avec {len(films_reduits)} films.")

# Étape 2 : Construire un graphe optimisé basé sur les genres
def construire_graphe_par_genre(films_reduits):
    """
    Construit un graphe où les nœuds sont des films reliés par des genres communs.
    """

    #Pour chaque Genre affecte les filme de ce genre
    index_genres = defaultdict(set)
    for _, row in films_reduits.iterrows():
        for genre in row['genres'].split('|'):
            index_genres[genre.strip().lower()].add(row['normalized_title'].lower())

    #Represente les relation entre les films par le genre Noeud=Film, Arc=Genre
    graphe = defaultdict(list)
    for films in index_genres.values():
        for film1 in films:
            for film2 in films:
                if film1 != film2:
                    graphe[film1].append(film2)
    return graphe

# Construire le graphe
graphe_films = construire_graphe_par_genre(films_reduits)
print(f"Graphe généré avec {len(graphe_films)} nœuds.")

# Étape 3 : Recherche par titre avec BFS
def bfs_recommendations(graphe, film_depart, films_reduits, profondeur_max=499):
    """
    Utilise BFS pour recommander des films ayant des genres similaires
    et qui appartiennent uniquement aux 500 films.
    """
    film_depart = film_depart.strip().lower()
    visite = set()
    file = deque([(film_depart, 0)])
    recommendations = []

    while file:
        film_courant, profondeur = file.popleft()
        if profondeur > profondeur_max:
            break

        if film_courant not in visite:
            recommendations.append(film_courant)
            visite.add(film_courant)

            for voisin in graphe.get(film_courant, []):
                if voisin not in visite:
                    file.append((voisin, profondeur + 1))

    # Filtrer et trier les films par rating décroissant
    recommendations_formatted = films_reduits[
        films_reduits['normalized_title'].str.lower().isin(recommendations)
    ].sort_values(by='average_rating', ascending=False)
    return recommendations_formatted[['title', 'genres', 'average_rating']]

# Étape 4 : Recherche par genres multiples
def rechercher_par_genres(films_reduits, genres):
    """
    Retourne tous les films correspondant à plusieurs genres donnés.
    """
    genres_lower = [genre.strip().lower() for genre in genres]
    all_genres = set('|'.join(films_reduits['genres']).lower().split('|'))

    # Vérifier si tous les genres existent
    genres_absents = [genre for genre in genres_lower if genre not in all_genres]
    if genres_absents:
        return f"Genres absents : {', '.join(genres_absents)}", None

    # Filtrer les films correspondant aux genres
    films_genres = films_reduits[
        films_reduits['genres'].apply(lambda x: all(genre in x.lower() for genre in genres_lower))
    ].sort_values(by='average_rating', ascending=False)
    return None, films_genres[['title', 'genres', 'average_rating']]

# Interface utilisateur
print("Bienvenue dans le système de recommandation de films !")
print("Choisissez une méthode de recherche :")
print("1. Recherche par titre")
print("2. Recherche par genres multiples")

choix = input("Entrez 1 pour le titre ou 2 pour les genres : ").strip()

if choix == "1":
    # Recherche par titre
    film_depart = input("Entrez le titre du film de départ (sans l'année) : ").strip()
    film_existe = films_reduits['normalized_title'].str.lower().isin([film_depart.lower()]).any()

    if film_existe:
        # Obtenir les recommandations
        resultats = bfs_recommendations(graphe_films, film_depart, films_reduits)
        if not resultats.empty:
            print(f"\nFilms similaires à '{film_depart}' (triés par rating) :")
            print(resultats)
        else:
            print(f"Aucun film similaire trouvé pour '{film_depart}'.")
    else:
        print(f"Film '{film_depart}' introuvable dans la base réduite. Vérifiez le titre et réessayez.")
elif choix == "2":
    # Recherche par genres multiples
    genres = input("Entrez les genres séparés par des virgules (ex: Drama, Comedy) : ").split(',')
    erreur, resultats = rechercher_par_genres(films_reduits, genres)

    if erreur:
        print(erreur)
    elif not resultats.empty:
        print(f"\nFilms correspondant aux genres '{', '.join(genres)}' (triés par rating) :")
        print(resultats)
    else:
        print(f"Aucun film trouvé pour les genres '{', '.join(genres)}'.")
else:
    print("Choix invalide. Veuillez entrer 1 ou 2.")


Base réduite générée avec 500 films.
Graphe généré avec 500 nœuds.
Bienvenue dans le système de recommandation de films !
Choisissez une méthode de recherche :
1. Recherche par titre
2. Recherche par genres multiples
Entrez 1 pour le titre ou 2 pour les genres : 1
Entrez le titre du film de départ (sans l'année) : Waiting to Exhale

Films similaires à 'Waiting to Exhale' (triés par rating) :
                                                 title  \
433                        What Happened Was... (1994)   
432  In the Realm of the Senses (Ai no corrida) (1976)   
48                                     Lamerica (1994)   
405                             Live Nude Girls (1995)   
121                   Awfully Big Adventure, An (1995)   
..                                                 ...   
271                     Stuart Saves His Family (1995)   
408                               House Party 3 (1994)   
150                        Love & Human Remains (1993)   
414                      

Algorithme de recherche A*

In [8]:
import pandas as pd
from collections import defaultdict, deque
import re
import heapq

# Charger les fichiers
movies_file = "drive/MyDrive/movies_reduced_to_500.xlsx"  # Chemin vers la table réduite des films
ratings_file = "drive/MyDrive/ratings_reduced_to_500.xlsx"  # Chemin vers la table réduite des ratings

# Lire les fichiers Excel
movies_data = pd.read_excel(movies_file, engine="openpyxl")
ratings_data = pd.read_excel(ratings_file, engine="openpyxl")

# Fonction pour extraire le titre sans l'année
def extraire_titre_sans_annee(titre):
    match = re.match(r"^(.*?)(?:\s\(\d{4}\))?$", titre)
    return match.group(1).strip() if match else titre.strip()

# Étape 1 : Réduire la base de données
def reduire_base_donnees(movies_data, ratings_data):
    """
    Combine les films et leurs notes pour créer une base cohérente.
    """
    popularite = ratings_data.groupby('movieId').agg(
        average_rating=('rating', 'mean'),
        num_ratings=('rating', 'count')
    ).reset_index()

    films_populaires = movies_data.merge(popularite, on='movieId')
    films_populaires['normalized_title'] = films_populaires['title'].apply(extraire_titre_sans_annee)
    return films_populaires[['movieId', 'title', 'genres', 'average_rating', 'num_ratings', 'normalized_title']]

# Réduire la base de données
films_reduits = reduire_base_donnees(movies_data, ratings_data)
print(f"Base réduite générée avec {len(films_reduits)} films.")

# Étape 2 : Construire un graphe optimisé basé sur les genres
def construire_graphe_par_genre(films_reduits):
    """
    Construit un graphe où les nœuds sont des films reliés par des genres communs.
    """
    index_genres = defaultdict(set)
    for _, row in films_reduits.iterrows():
        for genre in row['genres'].split('|'):
            index_genres[genre.strip().lower()].add(row['normalized_title'].lower())

    graphe = defaultdict(list)
    for films in index_genres.values():
        for film1 in films:
            for film2 in films:
                if film1 != film2:
                    graphe[film1].append(film2)
    return graphe

# Construire le graphe
graphe_films = construire_graphe_par_genre(films_reduits)
print(f"Graphe généré avec {len(graphe_films)} nœuds.")

# Étape 3 : Recherche par titre avec A*
def astar_recommendations(graphe, film_depart, films_reduits, profondeur_max=499):
    """
    Utilise l'algorithme A* pour recommander des films similaires avec une exploration optimisée.
    """
    film_depart = film_depart.strip().lower()
    visite = set()

    # File de priorité (min-heap)
    file_priorite = []
    heapq.heappush(file_priorite, (0, film_depart, 0))  # (coût total, film courant, profondeur)

    recommendations = []

    while file_priorite:
        cout_total, film_courant, profondeur = heapq.heappop(file_priorite)

        if profondeur > profondeur_max:
            break

        if film_courant not in visite:
            visite.add(film_courant)
            recommendations.append(film_courant)

            for voisin in graphe.get(film_courant, []):
                if voisin not in visite:
                    # Calculer \(g(n)\) (coût accumulé) et \(h(n)\) (heuristique)
                    g = profondeur + 1
                    h = calcul_heuristique(voisin, films_reduits)
                    heapq.heappush(file_priorite, (g + h, voisin, profondeur + 1))

    # Filtrer et trier les recommandations par note moyenne
    recommendations_formatted = films_reduits[
        films_reduits['normalized_title'].str.lower().isin(recommendations)
    ].sort_values(by='average_rating', ascending=False)
    return recommendations_formatted[['title', 'genres', 'average_rating']]

def calcul_heuristique(film, films_reduits):
    """
    Heuristique pour prioriser les films : ici, on utilise une combinaison de la popularité
    (nombre de notes) et de la note moyenne.
    """
    film_info = films_reduits[films_reduits['normalized_title'].str.lower() == film]
    if film_info.empty:
        return float('inf')  # Si le film est introuvable, pénalité élevée
    # Exemple : inverser la popularité (plus le score est élevé, plus \(h\) est faible)
    moyenne = film_info['average_rating'].iloc[0]
    nombre_notes = film_info['num_ratings'].iloc[0]
    return -moyenne * 0.7 + -nombre_notes * 0.3  # Ponderation des critères

# Étape 4 : Recherche par genres multiples
def astar_recommendations_with_genre(graphe, film_depart, films_reduits, genres=None, profondeur_max=499):
    """
    Utilise A* pour recommander des films similaires à un film donné, avec la possibilité de filtrer par genres.
    """
    film_depart = film_depart.strip().lower()
    visite = set()

    # Si des genres sont spécifiés, filtrer les films
    if genres:
        genres_lower = [genre.strip().lower() for genre in genres]
        films_reduits = films_reduits[
            films_reduits['genres'].apply(lambda x: all(genre in x.lower() for genre in genres_lower))
        ]
        if films_reduits.empty:
            print(f"Aucun film ne correspond aux genres '{', '.join(genres)}'.")
            return pd.DataFrame()

    # Construire un sous-graphe basé sur les films filtrés (si genres spécifiés)
    if genres:
        sous_graphe = {
            film: [voisin for voisin in voisins if voisin in films_reduits['normalized_title'].str.lower().tolist()]
            for film, voisins in graphe.items()
            if film in films_reduits['normalized_title'].str.lower().tolist()
        }
    else:
        sous_graphe = graphe

    # File de priorité (min-heap)
    file_priorite = []
    heapq.heappush(file_priorite, (0, film_depart, 0))  # (coût total, film courant, profondeur)

    recommendations = []

    while file_priorite:
        cout_total, film_courant, profondeur = heapq.heappop(file_priorite)

        if profondeur > profondeur_max:
            break

        if film_courant not in visite:
            visite.add(film_courant)
            recommendations.append(film_courant)

            for voisin in sous_graphe.get(film_courant, []):
                if voisin not in visite:
                    # Calculer \(g(n)\) (coût accumulé) et \(h(n)\) (heuristique)
                    g = profondeur + 1
                    h = calcul_heuristique(voisin, films_reduits)
                    heapq.heappush(file_priorite, (g + h, voisin, profondeur + 1))

    # Filtrer et trier les recommandations par note moyenne
    recommendations_formatted = films_reduits[
        films_reduits['normalized_title'].str.lower().isin(recommendations)
    ].sort_values(by='average_rating', ascending=False)
    return recommendations_formatted[['title', 'genres', 'average_rating']]

# Interface utilisateur
print("Bienvenue dans le système de recommandation de films !")
print("Choisissez une méthode de recherche :")
print("1. Recherche par titre")
print("2. Recherche par genres multiples")

choix = input("Entrez 1 pour le titre ou 2 pour les genres : ").strip()

if choix == "1":
    film_depart = input("Entrez le titre du film de départ (sans l'année) : ").strip()
    film_existe = films_reduits['normalized_title'].str.lower().isin([film_depart.lower()]).any()

    if film_existe:
        # Obtenir les recommandations avec A*
        resultats = astar_recommendations(graphe_films, film_depart, films_reduits)
        if not resultats.empty:
            print(f"\nFilms similaires à '{film_depart}' (triés par rating) :")
            print(resultats)
        else:
            print(f"Aucun film similaire trouvé pour '{film_depart}'.")
    else:
        print(f"Film '{film_depart}' introuvable dans la base réduite. Vérifiez le titre et réessayez.")
elif choix == "2":
    # Recherche par genres multiples
    genres = input("Entrez les genres séparés par des virgules (ex: Drama, Comedy) : ").split(',')
    genres = [g.strip() for g in genres if g.strip()]  # Nettoyer les entrées

    erreur, resultats = rechercher_par_genres(films_reduits, genres)
    if erreur:
        print(erreur)
    elif not resultats.empty:
        print(f"\nFilms correspondant aux genres '{', '.join(genres)}' (triés par rating) :")
        print(resultats)
    else:
        print(f"Aucun film trouvé pour les genres '{', '.join(genres)}'.")
else:
    print("Choix invalide. Veuillez entrer 1 ou 2.")

Base réduite générée avec 500 films.
Graphe généré avec 500 nœuds.
Bienvenue dans le système de recommandation de films !
Choisissez une méthode de recherche :
1. Recherche par titre
2. Recherche par genres multiples
Entrez 1 pour le titre ou 2 pour les genres : 1
Entrez le titre du film de départ (sans l'année) : Waiting to Exhale

Films similaires à 'Waiting to Exhale' (triés par rating) :
                                                 title  \
433                        What Happened Was... (1994)   
432  In the Realm of the Senses (Ai no corrida) (1976)   
48                                     Lamerica (1994)   
405                             Live Nude Girls (1995)   
121                   Awfully Big Adventure, An (1995)   
..                                                 ...   
271                     Stuart Saves His Family (1995)   
408                               House Party 3 (1994)   
150                        Love & Human Remains (1993)   
414                      